<center>
<img style="float: center;" src="images/CI_horizontal.png" width="400">
</center>
<center>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Website</a>
    </span>
</center>

<center> Julia Lane, Clayton Hunter, Brian Kim, Benjamin Feder, Ekaterina Levitskaya, Tian Lou, Lisa Osorio-Copete. 
</center>

**_Disclosure Review Examples & Exercises_**

This notebook provides you with information on how to prepare research output for disclosure control. It outlines how to prepare different kind of outputs before submitting an export request and gives you an overview of the information needed for disclosure review. _Please read through the entire notebook because it will separately discuss different types of outputs that will be flagged in the disclosure review process._

In [ ]:
#database interaction imports
library(DBI)
library(RPostgreSQL)

# for data manipulation/visualization
library(tidyverse)

# scaling data, calculating percentages, overriding default graphing
library(scales)

In [ ]:
# create an RPostgreSQL driver
drv <- dbDriver("PostgreSQL")

# connect to the database
con <- dbConnect(drv,dbname = "postgresql://stuffed.adrf.info/appliedda")

# General Remarks on Disclosure Review

## Files you can export
In general, you can export any kind of file format. However, most researchers typically export tables, graphs, regression outputs and aggregated data. Thus, we ask you to export one of these types, which implies that every result you would like to export needs to be saved in either .csv, .txt or graph format.

## Jupyter notebooks are only exported to retrieve code
Unfortunately, you can't export results in a Jupyter notebook. Doing disclosure reviews on output in Jupyter notebooks is too burdensome for us. Jupyter notebooks will only be exported when the output is deleted for the purpose of exporting code. **This does not mean that you won't need your Jupyter notebooks during the export process.** 

## Documentation of code is important
During the export process, we ask you to provide the code for every output you would like to export. It is important for the ADRF staff to have the code to better understand what you exactly did. Understanding how research results are created is important in understanding your research output. Thus, it is important to document every step of your analysis in your Jupyter notebook. 

## General rules to keep in mind
A more detailed description of the rules for exporting results can be found on the class website. This is just a quick overview. You should go to the class website and read the entire guidelines (link below) before preparing your files for export. 
- The disclosure review is based on the underlying observations of your study. **Every statistic you want to export must be based on at least 10 individuals. When reporting any employment statistics, the statistics must be based on 3 firms and employment in no one firm comprises more than 80% of the associated group.** You must show the disclosure review team that every statistic you wish to export is based on those numbers by providing the associated counts/percentages in your input file. 
- Document your code so the reviewer can follow your data work. Assessing re-identification risks highly depends on the context. Therefore, it is important that you provide context info with your analysis for the reviewer. When making comments in the code, make sure not to use any individual statistic (e.g. the mean is ...).
- Save the requested output with the corresponding code in your input and output folder. Make sure the code is executable. The code should exactly produce the output you requested.
- If you are exporting powerpoint slides that show project results, you have to provide the code which produces the output in the slide.
- Please export results only when they are final and you need them for your presentation or final project report.

## To-Do:
Read through the **documentation** link: adrf.readthedocs.io/en/latest/export_of_results/guidelines.html#documentation

# Disclosure Review Walkthrough

You will reconstruct the statistics and visualizations created in the Data Exploration and Data Visualization notebooks and prepare them to pass the disclosure review process.

## Counts

Recall the first guiding question in the "Understanding Our Graduates" [section](03_Data_Exploration.ipynb/#The-TANF-Experience) of the Data Exploration notebook:

- How many individuals are in the 2016Q4 cohort?

Before we re-answer this question, we need to read this cohort into R.

In [ ]:
# read cohort into R
qry <- "
select * 
from ada_tdc_2020.cohort_2016
"
df_2016 <- dbGetQuery(con, qry)

# see df_2016
head(df_2016)

From here, we found the number of graduates using `nrow()`. Because the desired statistics for export are not from any of the employment tables, we just need to show that the counts within all of these groups are at least 10. In this case, since we do not have any subgroups, we just need to show that the number of individuals in the 2016Q4 cohort is at least 10.

Otherwise, as mentioned above, you would need to show that each count is comprised of at least 3 firms and employment in no one firm comprises more than 80% of the group to receive your export.

In [ ]:
# because each row is a unique ssn, can just get the count of rows
nrow(df_2016)

To export this output as a csv, you can use the `write_csv()` function as long as you designate the file path and the name of the .csv. Here, we will call the file `count_2016_cohort.csv` (the more descriptive the name of the file, the easier it is for the Coleridge Initiative's export team to review).

> In the file path, as long as you assign the `user` object to your ADRF username, the csv will be saved in your home folder. Review the `sprintf` section in the Data Exploration [notebook](03_Data_Exploration.ipynb/#sprintf) if you are unsure as to how `sprintf` can be used.

In [ ]:
# change to your username for saving files
user <- 'benjaminfeder'

In [ ]:
# save as csv
nrow(df_2016) %>%
    as.data.frame() %>%
    write_csv(sprintf('/nfshome/%s/count_2016_cohort.csv', user))

Quickly, we will walk through preparing the subsequent question for disclosure review, as it contains subgroups. The question is:
- What is the age breakdown of the cohort?

To answer this question, we used the following code (now combined into one cell):

In [ ]:
# save to ages
ages <- df_2016 %>%
    mutate(age = as.numeric(rep_year) - dob_yr) %>%
    select(rep_year, dob_yr, age)

# create age groups
ages <- ages %>%
    mutate(age_group = case_when(
        age < 18 ~ "0-17",
        between(age, 18, 39) ~ "18-39",
        between(age, 40, 59) ~ "40-59",
        age >= 60 ~ "60+"
    )
          )

# see counts by age group
ages %>%
    count(age_group)

In this example, our subgroup is created by the `age_group` variable. As long as each of these counts within `age_group` contains at least 10 individuals, the export will pass disclosure review. However, you can see that within the cohort, the count for the REDACTED group would not pass disclosure review, so we can either disregard the group in the export, or combine two of the groups. Here, we will simply disregard these individuals in the export.

In [ ]:
ages %>%
    filter(age_group != 'REDACTED') %>%
    count(age_group)

This data frame will now pass disclosure review.

In [ ]:
# save as csv
ages %>%
    filter(age_group != 'REDACTED') %>%
    count(age_group) %>%
    write_csv(sprintf('/nfshome/%s/count_2016_cohort_by_age_group.csv', user))

### Percentages

A subset of working with counts, with any reported percentages, you must provide underlying counts of the numerators and denominators for each group involved in your desired export. To illustrate this notion, we will work through another example from the "The TANF Experience" [section](03_Data_Exploration.ipynb/#The-TANF-Experience) in the Data Exploration notebook:

- Are we seeing a concentration of lengthy spells in specific regions?

Recall the code below that we used to find the answer to this question.

In [ ]:
# assign better looking 90th percentile output to percentile
percentile <- df_2016 %>%
    summarize('.9' = quantile(tanf_spell_months, .9))

# assign to variable "lens"
lens <- df_2016%>%
    select(ssn, county, tanf_spell_months) %>%
    mutate(rel_length = ifelse(tanf_spell_months > percentile$'.9', 'long', 'not long'))

# find count and proportion of "long" by county sorted by highest proportion
lens %>%
    group_by(county, rel_length) %>%
    summarize(n=n()) %>%
    mutate(Proportion = n/sum(n)) %>%
    ungroup() %>%
    filter(rel_length == 'long') %>%
    arrange(desc(Proportion)) %>%
    # we don't need to see rel_length column
    select(-rel_length) %>%
    head()

Let's say we want to export the proportion of lengthy spells by county. In this case, we need to make sure that the count of the number of individuals with lengthy spells, as well as the total number of individuals with spells in the county, are both at least 10. We already have the number of individuals with length spells by county, as each row in `lens` pertains to one individual. Therefore, we just need to find the number of total spells by county, which we have already found in `sum(n)`. Let's add that to our data frame as `total`.

In [ ]:
# add in total individuals by county
lens %>%
    group_by(county, rel_length) %>%
    summarize(n=n()) %>%
    mutate(
        Proportion = n/sum(n),
        total = sum(n)
    ) %>%
    ungroup() %>%
    filter(rel_length == 'long') %>%
    # we don't need to see rel_length column
    select(-rel_length) %>%
    head()

Now, we can simply `filter()` for counties with at least 10 individuals with lengthy spells (`n`) and 10 individuals with spells (`total`).

In [ ]:
# filter to fit disclosure review guidelines
lens %>%
    group_by(county, rel_length) %>%
    summarize(n=n()) %>%
    mutate(
        Proportion = n/sum(n),
        total = sum(n)
    ) %>%
    ungroup() %>%
    filter(
        rel_length == 'long',
        n >= 10,
        total >= 10
    ) %>%
    # we don't need to see rel_length column
    select(-rel_length)

Unfortunately, given these disclosure review constraints, only REDACTED counties would be exported. Perhaps, if you run into this problem, you can alter this calculation by potentially increasing the unit of analysis (`county`) or by changing the definition of a lengthy spell. Regardless, let's save this resulting data frame as a csv.

> Because the counts are again in the same data frame as the one you would like to export, you do not need to save an additional data frame to show proof of the counts of individuals per county.

In [ ]:
# save as csv
lens %>%
    group_by(county, rel_length) %>%
    summarize(n=n()) %>%
    mutate(
        Proportion = n/sum(n),
        total = sum(n)
    ) %>%
    ungroup() %>%
    filter(
        rel_length == 'long',
        n >= 10,
        total >= 10
    ) %>%
    # we don't need to see rel_length column
    select(-rel_length) %>%
    write_csv(sprintf('/nfshome/%s/prop_lengthy_spells_by_county.csv', user))

## Fuzzy percentiles

Under no circumstances will you be able to export any percentiles, regardless of if the unit of analysis is directly subject to disclosure review. To get a sense of percentiles in the data, you can export fuzzy percentiles, which can be created by finding the average of two true percentiles. 

For example, you can export a fuzzy median by finding the average of the true 45th and 55th percentiles. We will provide an example of preparing data for export by walking through another example from the Data Exploration notebook, this time from the "Post-TANF Employment Outcomes" [section](03_Data_Exploration.ipynb/#Post-TANF-Employment-Outcomes):

- What is the distribution of quarterly wages for the 2016Q4 cohort?

Since we will be focusing on the 2016Q4 cohort's earnings, we will load in the `cohort_2016_earnings` table from the `ada_tdc_2020` schema.

In [ ]:
# read cohort's wages into R
qry = "
select *
from ada_tdc_2020.cohort_2016_earnings
"
df_2016_wages <- dbGetQuery(con, qry)

Initially, to answer this question, we used the following code:

In [ ]:
# save quarterly earnings as wages_quarter
wages_quarter <- df_2016_wages %>%
    group_by(ssn, quarter) %>%
    summarize(quarterly_wages = sum(wages)) %>%
    ungroup()

In [ ]:
# more nuanced look at quarterly earnings distribution
wages_quarter %>%
    summarize('.1' = quantile(quarterly_wages, .1),
              '.25' = quantile(quarterly_wages, .25),
              '.5' = quantile(quarterly_wages, .5),
              '.75' = quantile(quarterly_wages, .75),
              '.9' = quantile(quarterly_wages, .9)
             )

This data frame is not ready for export for four reasons:
    
1. These numbers represent exact percentiles and will not be accepted in the export process
2. We do not have underlying counts per each group (groups not applicable, but still need counts of individuals)
3. No evidence of lack of employer dominance
4. We do not have underlying employer counts

Let's first fuzzy these percentiles. We can do this by finding the average of our two true percentile values equidistant from the fuzzy percentile in question.

In [ ]:
# more nuanced look at quarterly earnings distribution
wages_quarter %>%
    summarize('.1' = quantile(quarterly_wages, .1),
              '.25' = quantile(quarterly_wages, .25),
              '.5' = quantile(quarterly_wages, .5),
              '.75' = quantile(quarterly_wages, .75),
              '.9' = quantile(quarterly_wages, .9)
             )

Now we have found fuzzy percentiles for our desired outputs. To finalize this aspect of the export, we need to add in the underlying counts within each of the groups (only one group in this case), which we can do easily by adding `n_distinct(ssn)` to our `summarize()` function.

In [ ]:
# fuzzy quantiles with counts
wages_quarter %>%
    summarize('fuzzy 10' = (quantile(quarterly_wages, .05) + quantile(quarterly_wages, .15))/2,
              'fuzzy 25' = (quantile(quarterly_wages, .20) + quantile(quarterly_wages, .30))/2,
              'fuzzy 50' = (quantile(quarterly_wages, .45) + quantile(quarterly_wages, .55))/2,
              'fuzzy 75' = (quantile(quarterly_wages, .70) + quantile(quarterly_wages, .80))/2,
              'fuzzy 90' = (quantile(quarterly_wages, .8) + quantile(quarterly_wages, .95))/2,
              count = n_distinct(ssn)
             )

Since we have provided underlying counts of individuals in the same data frame we would like to export, we do not need to additionally export a data frame of the underlying individual counts. However, because we are working with earnings and employment data, we need to show there are at least three employers that constitute the wage calculations within each group, as well as a lack of a single-employer dominance. We will define employer dominance as an employer providing at least 80 percent of the weight in the calculation. Therefore, if a group's employment relies on that ratio or greater from one employer, we will need to redact it in order to receive the export.

Let's first find the number of employers involved in the calculations above, as it is simply the number of employers in `df_2016_wages`.

In [ ]:
# find number of employers
df_2016_wages %>% 
    summarize(
        n_employers = n_distinct(uiacct)
    )

Finally, to verify the lack of employer dominance, you can work through the following steps:
- Count the instances of the employers within each grouping variable, if a grouping variable exists
- Use `group_by()` to group the grouping variable (if applicable)
- Calculate the proportion of entries per grouping variable for each employer
- Select the most common employer using `top_n()`

> In this case, we don't need to specify the variable on which `top_n()` will be sorted because sorting by `n` or `prop` in this example will yield the same result.

In [ ]:
# see employer dominance
df_2016_wages %>% 
    count(uiacct) %>%
    mutate(prop = n/sum(n)) %>%
    top_n(1)

Now that we have verified high enough individual counts, created fuzzy percentiles, and analyzed the employers of these individuals, we can simply export the data frame in question by piping our output into `write_csv()`.

In [ ]:
# save as csv
wages_quarter %>%
    summarize('fuzzy 10' = (quantile(quarterly_wages, .05) + quantile(quarterly_wages, .15))/2,
              'fuzzy 25' = (quantile(quarterly_wages, .20) + quantile(quarterly_wages, .30))/2,
              'fuzzy 50' = (quantile(quarterly_wages, .45) + quantile(quarterly_wages, .55))/2,
              'fuzzy 75' = (quantile(quarterly_wages, .70) + quantile(quarterly_wages, .80))/2,
              'fuzzy 90' = (quantile(quarterly_wages, .8) + quantile(quarterly_wages, .95))/2,
              count = n_distinct(ssn)
             ) %>%
    write_csv(sprintf('/nfshome/%s/fuzzy_quarterly_wages.csv', user))

We will save our employer counts and dominance tests for export as well.

In [ ]:
# save employer dominance
df_2016_wages %>% 
    count(uiacct) %>%
    mutate(prop = n/sum(n)) %>%
    top_n(1) %>%
    write_csv(sprintf('/nfshome/%s/fuzzy_quarterly_wages_employer_dominance.csv', user))

In [ ]:
# save number of employers
df_2016_wages %>% 
    count(uiacct) %>%
    mutate(prop = n/sum(n)) %>%
    top_n(1) %>%
    write_csv(sprintf('/nfshome/%s/fuzzy_quarterly_wages_employer_counts.csv', user))

## Visualizations

The same disclosure controls we have covered to this point for counts and other statistics apply for visualizations too, as you will need to provide underlying counts by groups for each of your visualizations. Here, we will provide examples for disclosure-proofing different types of visualizations based on the examples provided in the Data Visualization notebook. We will start with our first example from the notebook: a boxplot.

### Boxplots

Disclosure-proofing boxplots can be quite tricky since you cannot export a boxplot with true percentiles or outlines. Instead, you must create a boxplot using fuzzy percentiles by will user-inputting the fuzzy values for our boxplot. Let's see how this works through disclosure-proofing the [third visualization](05_Data_Visualization.ipynb/#Distribution-of-spell-lengths-in-the-cohort) in the Data Visualization notebook as a boxplot:

- Creating a boxplot of the distribution of spell lengths for the 2016Q4 cohort

Below is the code we used in the notebook, just adapted to create a boxplot instead of a histogram.

In [ ]:
# Full code for the plot
ggplot(df_2016, aes(x='', y=tanf_spell_months)) + 
geom_boxplot() +
ggtitle('Most Common Spell Length in 2016Q4 Cohort: REDACTED') +
xlab('TANF spell months') +
ylab('Number of individuals') +
theme(text=element_text(size=12, face="bold"))  +
labs(caption = 'Source: Indiana TANF data') +
theme(plot.caption = element_text(hjust=0))

Let's isolate the components we need to create a safe boxplot:

- fuzzy 25th percentile
- fuzzy 75th percentile
- fuzzy median (50th percentile)
- fuzzy minimum
- fuzzy maximum
- no outliers

We can take these separate components of the boxplot (outside of the no outliers) and manually input them into the `ggplot(aes()` call. Therefore, we will create a data frame containing each of these components, and then pipe it into `ggplot()` accordingly. First, let's generate fuzzy 25th, 50th and 75th percentiles and save this to `stats`.

In [ ]:
# find 25, 50 and 75 fuzzy percentiles
stats <- df_2016 %>%
    summarize(
        'fuzzy_25' = (quantile(tanf_spell_months, .20) + quantile(tanf_spell_months, .30))/2,
        'fuzzy_50' = (quantile(tanf_spell_months, .45) + quantile(tanf_spell_months, .55))/2,
        'fuzzy_75' = (quantile(tanf_spell_months, .70) + quantile(tanf_spell_months, .80))/2
        )
# see stats
stats

To handle fuzzy minimum and maximum values, we will _first_ calculate the cutoff values (in both directions) to determine if an institution would be viewed as an outlier by its amount of graduates. _Second_, we will `filter()` out all institutions whose counts of graduates are outside of this bound, and then _third_, and finally, we will take the average of the counts of the institutions with the lowest and the highest counts within this bound to find the fuzzy minimum and maximum values, respectively.

Let's start with our first step of finding the minimum and maximum cutoff values. We'll add these to `stats` so we can refer to them easily in the future.

In [ ]:
# find minimum and maximum cutoff values
stats <- stats %>%
    mutate(
        fuzzy_min_cutoff = (fuzzy_25 - 1.5*(fuzzy_75 - fuzzy_25)),
        fuzzy_max_cutoff = (fuzzy_75 + 1.5*(fuzzy_75 - fuzzy_25))
    )
# see stats
stats

Next, we will filter our original data frame, `df_2016`, to only individuals whose `tanf_spell_months` remain within the `fuzzy_min_cutoff` and `fuzzy_max_cutoff` values. We will call this data frame `new_df`.

In [ ]:
# find df_2016 with no outliers as per fuzzy min and max cutoffs
new_df <- df_2016 %>%
    filter((tanf_spell_months > stats$fuzzy_min_cutoff) & (tanf_spell_months < stats$fuzzy_max_cutoff))

Finally, we will find our `fuzzy_min` and `fuzzy_max` values by taking the average of the two `tanf_spell_months` smallest values, as well as the two `tanf_spell_months` with the two largest values `new_df`. We will save these values as variables in `stats`.

In [ ]:
# find fuzzy min
new_max <- new_df %>%
    arrange(desc(tanf_spell_months)) %>%
    head(2) %>%
    summarize(m = mean(tanf_spell_months))

# find fuzzy max
new_min <- new_df %>%
    arrange(tanf_spell_months) %>%
    head(2) %>%
    summarize(m = mean(tanf_spell_months))

# save fuzzy_min and fuzzy_max to stats
stats <- stats %>%
    mutate(
        fuzzy_min = new_min$m,
        fuzzy_max = new_max$m
    )
# see stats
stats

Now, let's pipe `stats` into our `ggplot()` call, feeding in everything besides the `fuzzy_min_cutoff` and `fuzzy_max_cutoff` values.

In [ ]:
stats %>%    
    ggplot(aes(x="", ymin = fuzzy_min, lower = fuzzy_25, middle = fuzzy_50, upper = fuzzy_75, ymax = fuzzy_max)) +
    geom_boxplot(stat="identity") +
    ggtitle('Most Common Spell Length in 2016Q4 Cohort: REDACTED') +
    xlab('TANF spell months') +
    ylab('Number of individuals') +
    theme(text=element_text(size=12, face="bold"))  +
    labs(caption = 'Source: Indiana TANF data') +
    theme(plot.caption = element_text(hjust=0))

Thus, if you were to put this code all together in one code cell, it would look like the code cell below.

> We added `ggsave()` at the end, as this will allow you to save your most recent visualization as a PDF.

In [ ]:
# make fuzzy boxplot
# get fuzzy 25, 50, 75 and min/max cutoffs
stats <- df_2016 %>%
    summarize(
        'fuzzy_25' = (quantile(tanf_spell_months, .20) + quantile(tanf_spell_months, .30))/2,
        'fuzzy_50' = (quantile(tanf_spell_months, .45) + quantile(tanf_spell_months, .55))/2,
        'fuzzy_75' = (quantile(tanf_spell_months, .70) + quantile(tanf_spell_months, .80))/2
        ) %>%
    # find min and max cutoff values
   mutate(
        fuzzy_min_cutoff = (fuzzy_25 - 1.5*(fuzzy_75 - fuzzy_25)),
        fuzzy_max_cutoff = (fuzzy_75 + 1.5*(fuzzy_75 - fuzzy_25))
    )
# find df_2016 with no outliers as per fuzzy min and max cutoffs
new_df <- df_2016 %>%
    filter((tanf_spell_months > stats$fuzzy_min_cutoff) & (tanf_spell_months < stats$fuzzy_max_cutoff))

# find fuzzy min
new_max <- new_df %>%
    arrange(desc(tanf_spell_months)) %>%
    head(2) %>%
    summarize(m = mean(tanf_spell_months))

# find fuzzy max
new_min <- new_df %>%
    arrange(tanf_spell_months) %>%
    head(2) %>%
    summarize(m = mean(tanf_spell_months))

# plot same graph
stats %>%
    mutate(
        fuzzy_min = new_min$m,
        fuzzy_max = new_max$m
    ) %>%  
    ggplot(aes(x="", ymin = fuzzy_min, lower = fuzzy_25, middle = fuzzy_50, upper = fuzzy_75, ymax = fuzzy_max)) +
    geom_boxplot(stat="identity") +
    ggtitle('Most Common Spell Length in 2016Q4 Cohort: REDACTED') +
    xlab('TANF spell months') +
    ylab('Number of individuals') +
    theme(text=element_text(size=12, face="bold"))  +
    labs(caption = 'Source: Indiana TANF data') +
    theme(plot.caption = element_text(hjust=0))

# save plot
ggsave(sprintf("/nfshome/%s/fuzzy_boxplot_grads.pdf", user))

### Histogram

The thought process behind exporting histograms is very similar to the ones for other visualizations, as you must provide verification that each group (or bin, in this case) follows the disclosure review guidelines. While this may seem like a simple idea, it can sometimes require a bit of manipulation.

We will try preparing a histogram representing the distribution of TANF spell lengths, starting with the default settings of `geom_histogram()`.

> Note: If you choose to use a density plot, you do not need to provide counts per bin because there are no bins - you just need to provide the underlying counts per line.

In [ ]:
# default geom_histogram settings
df_2016 %>%
    ggplot(aes(x=tanf_spell_months)) +
    geom_histogram()

Although the count is naturally displayed on the vertical axis, the counts within each bin may not always be clear. We can add the `stat_bin()` layer to `geom_histogram()` to display the counts per bin.

In [ ]:
# displaying counts per bin
df_2016 %>%
    ggplot(aes(x=tanf_spell_months)) +
    geom_histogram() + 
    stat_bin(aes(y=..count.., label= ..count..), geom="text", vjust = 0)

We can clearly see that these bins do not all have at least 10 observations. We can approach this solution in a variety of ways, but in this notebook, we will adjust the number of bins. Let's try switching the number of bins from 30 (the default setting) to 15 in both the `geom_histogram()` and `stat_bin()` calls.

> Note: You can also adjust the edges of the bins by adding the `breaks` argument to both the `geom_histogram()` and then `stat_bin()` calls. However, in this example, since there are a lot of observations that can potentially fall into different bins, we will try to adjust the number of bins first.

In [ ]:
# displaying counts per bin with 15 bins
df_2016 %>%
    ggplot(aes(x=tanf_spell_months)) +
    geom_histogram(bins=15) + 
    stat_bin(aes(y=..count.., label= ..count..), geom="text", vjust = 0, bins=15)

Although we are quite close, there are still some bins that would not pass the disclosure controls. Let's try 13 bins.

In [ ]:
# displaying counts per bin with 13 bins
df_2016 %>%
    ggplot(aes(x=tanf_spell_months)) +
    geom_histogram(bins=13) + 
    stat_bin(aes(y=..count.., label= ..count..), geom="text", vjust = 0, bins=13)

Since each bin consists of at least 10 observations, you can then save this visualization for export using `ggsave()`.

In [ ]:
# save viz for export
df_2016 %>%
    ggplot(aes(x=tanf_spell_months)) +
    geom_histogram(bins=13) + 
    stat_bin(aes(y=..count.., label= ..count..), geom="text", vjust = 0, bins=13)

ggsave(sprintf('/nfshome/%s/spell_length_histogram.pdf', user))

Due to the number of bins, you may find this visualization unclear. As a comparison, you can look at a density plot, as it may be better suited export-wise or a density plot to best represent the general distribution of the data.

In [ ]:
# displaying counts per bin
df_2016 %>%
    ggplot(aes(x=tanf_spell_months)) +
    geom_density()

In this case, you would just need to show that there are at least 10 different individuals in the underlying data used to create this density plot. First, though, let's save this visualization.

In [ ]:
# save viz for export
df_2016 %>%
    ggplot(aes(x=tanf_spell_months)) +
    geom_density()

ggsave(sprintf('/nfshome/%s/spell_length_density.pdf', user))

In [ ]:
# save number of unique individuals
df_2016 %>%
    summarize(n = n_distinct(ssn)) %>%
    write_csv(sprintf('/nfshome/%s/spell_length_density_plot_counts.csv', user))

## Barplot

As we walk through an example of preparing a barplot for disclosure review using wage record data, recall the example in the Data Visualization [notebook](05_Data_Visualization.ipynb/#Most-popular-industries-with-the-highest-average-wages) of visualizing the average quarterly earnings for the most popular industries of employment for the 2016 cohort. Let's prepare the original visualization (created with the code below) for export.

In [ ]:
# save most popular naics as pop_naics
pop_naics <- df_2016_wages %>%
    group_by(naics_3_digit) %>%
    summarize(num = n_distinct(ssn)) %>% 
    arrange(desc(num))  %>%
    slice(1:10)     # choose top 10 industries

# get wages for top 10 industries
wages_pop_naics <- df_2016_wages %>%
    filter(naics_3_digit %in% pop_naics$naics_3_digit) %>%
    select(ssn, wages, naics_3_digit, quarter)

# save to quarterly_naics
quarterly_naics <- wages_pop_naics %>%
    group_by(ssn, quarter, naics_3_digit) %>%
    summarize(tot_wages = sum(wages)) %>%
    ungroup()

# find average quarterly wages by industry and include number of people employed at least one quarter in each industry
top_mean_wages <- quarterly_naics %>%
                group_by(naics_3_digit) %>%
                summarize(avg_wages = mean(tot_wages),
                         num_ssns = n_distinct(ssn)) %>%
                arrange(desc(num_ssns)) 

# read naics_2017 table into R as naics
qry = '
select *
from public.naics_2017
'
naics = dbGetQuery(con, qry)

# get industry names using left join with "naics" table, like we did above
top_mean_wages <- top_mean_wages %>% 
    left_join(naics, by=c('naics_3_digit' = 'naics_us_code')) %>%
    select(naics_us_title, avg_wages)

# Full code for the plot
ggplot(top_mean_wages, aes(x= reorder(naics_us_title, -avg_wages), y=avg_wages)) +
geom_bar(stat = 'identity', fill = 'blue') +
ggtitle('REDACTED - highest average wages (2016Q4)') +
xlab('Industry') +
ylab('Average wages') +
theme(text=element_text(size=13, face="bold"))  +
labs(caption = 'Source: Indiana TANF, UI Wage data') +
theme(plot.caption = element_text(hjust=0)) +
theme(axis.text.x = element_text(angle=90))  

We cannot export the above visualization just yet for the following reasons:
- No counts of the individuals within each industry in the underlying data
- Because we are working with wage data, we need to make sure that each industry is associated with at least 10 employers, as well as proof of the lack of employer dominance within each industry

Let's first find the number of individuals employed within each industry. In the code used to generate the visualization, we can find the number of individuals within each of these industries using the data frame `quarterly_naics`.

In [ ]:
# find number of distinct ssns per industry
quarterly_naics %>%
    group_by(naics_3_digit) %>%
    summarize(num_ssns = n_distinct(ssn))

Since the visualization does not contain `naics_3_digit`, but rather the titles corresponding to these three-digit NAICS codes, let's find the proper titles using the `naics` data frame. We can do so using a similar `left_join()` as we did in the code to generate the original visualization.

In [ ]:
# match to naics titles
quarterly_naics %>%
    group_by(naics_3_digit) %>%
    summarize(num_ssns = n_distinct(ssn)) %>%
    left_join(naics, by=c('naics_3_digit'='naics_us_code')) %>%
    select(naics_us_title, num_ssns)

As mentioned above, we also need to find some more information on the employers within these industries, both in terms of counts and dominance. To find the counts of the employers per industry, we need to go a bit further back in the code since `quarterly_naics` does not contain employer-level information. Here, we need to start with `df_2016_wages` and then find the number of unique employers within these popular industries.

In [ ]:
# find number of employers per these industries
df_2016_wages %>%
    filter(naics_3_digit %in% pop_naics$naics_3_digit) %>% 
    group_by(naics_3_digit) %>%
    summarize(num_employers = n_distinct(uiacct))

Like before, let's add in the naics titles so the disclosure control team will know to which naics code each title corresponds.

In [ ]:
# add in naics titles
df_2016_wages %>%
    filter(naics_3_digit %in% pop_naics$naics_3_digit) %>% 
    group_by(naics_3_digit) %>%
    summarize(num_employers = n_distinct(uiacct)) %>%
    left_join(naics, by = c('naics_3_digit'='naics_us_code')) %>%
    select(naics_us_title, num_employers)

Finally, let's find out if there was any employer dominance within each of these subgroups. Here, because we are concerned with the amount of rows corresponding to each `uiacct` within the industries, we will count the number of occurrences of each `uiacct` within each industry, and then find the proportion of rows corresponding to each `uiacct` by industry.

In [ ]:
# calculate employer dominance
df_2016_wages %>%
    filter(naics_3_digit %in% pop_naics$naics_3_digit) %>% 
    count(naics_3_digit, uiacct) %>%
    group_by(naics_3_digit) %>%
    mutate(prop=n/sum(n)) %>%
    top_n(1) %>%
    select(naics_3_digit, prop)

Finally, let's add in the corresponding titles to each three-digit NAICS code.

In [ ]:
# add in naics titles
df_2016_wages %>%
    filter(naics_3_digit %in% pop_naics$naics_3_digit) %>% 
    count(naics_3_digit, uiacct) %>%
    group_by(naics_3_digit) %>%
    mutate(prop=n/sum(n)) %>%
    top_n(1) %>%
    ungroup() %>% 
    select(naics_3_digit, prop) %>%
    left_join(naics, by = c('naics_3_digit'='naics_us_code')) %>%
    select(naics_us_title, prop)

Since all of counts of individuals and employers are at least 10 for each subgroup and there is no employer dominance within any of the subgroups, you can safely export this visualization. To finish it off, let's save all of these outputs in their respective file formats.

In [ ]:
# write number of individuals by industry to csv
quarterly_naics %>%
    group_by(naics_3_digit) %>%
    summarize(num_ssns = n_distinct(ssn)) %>%
    left_join(naics, by=c('naics_3_digit'='naics_us_code')) %>%
    select(naics_us_title, num_ssns) %>%
    write_csv(sprintf('/nfshome/%s/avg_earnings_pop_naics_barplot_individual_counts.csv', user))

In [ ]:
# write number of employers to csv
df_2016_wages %>%
    filter(naics_3_digit %in% pop_naics$naics_3_digit) %>% 
    group_by(naics_3_digit) %>%
    summarize(num_employers = n_distinct(uiacct)) %>%
    left_join(naics, by = c('naics_3_digit'='naics_us_code')) %>%
    select(naics_us_title, num_employers) %>%
    write_csv(sprintf('/nfshome/%s/avg_earnings_pop_naics_barplot_emp_counts.csv', user))

In [ ]:
# write employer dominance to csv
df_2016_wages %>%
    filter(naics_3_digit %in% pop_naics$naics_3_digit) %>% 
    count(naics_3_digit, uiacct) %>%
    group_by(naics_3_digit) %>%
    mutate(prop=n/sum(n)) %>%
    top_n(1) %>%
    ungroup() %>% 
    select(naics_3_digit, prop) %>%
    left_join(naics, by = c('naics_3_digit'='naics_us_code')) %>%
    select(naics_us_title, prop) %>%
    write_csv(sprintf('/nfshome/%s/avg_earnings_pop_naics_barplot_emp_dominance.csv', user))

In [ ]:
# save for export
ggsave(sprintf('/nfshome/%s/avg_earnings_pop_naics_barplot.pdf', user))

## Machine Learning

You should treat exporting clusters as you would with any other grouping variable, as each cluster must satisfy a minimum number of individuals and (when applicable) employers to pass disclosure control.

## Reminders
Every single item you wish to export, regardless of whether it is a .csv, .pdf, .png, or something else, must have corresponding proof in your input file to show that every group used to create this statistic followed our disclosure review rules.

Additionally, if you are exporting employer-level characteristics, you must also show a lack of employer dominance as well as a count of at least three employers per group.